In [1]:
!python -m spacy download pt_core_news_sm
!pip install googletrans==4.0.0-rc1
!pip install enelvo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 106.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.0 MB/s eta 

In [2]:
from google.colab import drive
from googletrans import Translator
from enelvo.normaliser import Normaliser
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import SimpleRNN, Dense, Concatenate, Input, LSTM, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support
import tensorflow as tf
import re
import numpy as np
import pandas as pd
import spacy
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


# Pré-processamento

## 0) Base de Dados

In [27]:
norm = Normaliser(tokenizer='readable')
chats_clients = pd.read_csv('/content/drive/Shareddrives/grupo3moshi/augmented_data.csv')
chats_clients.head()

,No,Intencao,Pergunta,Resposta
0,1,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d..."
1,2,Como fazer remessa,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f..."
2,3,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ..."
3,4,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...
4,5,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...


In [28]:
chats_clients.fillna('erro ao processar a pergunta', inplace=True)

In [29]:
nlp_pt = spacy.load("pt_core_news_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [30]:
def correct_grammar(text):
  return norm.normalise(text)

## 1) Word2Vec

O processo de Word2Vec é uma técnica de aprendizado de máquina com o objetivo de vetorizar palavras almejando preservar o contexto, convertendo o resultado para a forma numérica, para que os dados possam ser utilizados em modelos de aprendizado de máquina.

O Word2Vec pode possuir 2 arquiteturas, sendo elas:

- Skip-Gram: rede tenta prever as palavras de contexto ao redor de uma palavra central
- CBOW: rede tenta prever a palavra central com base no contexto ao redor da palavra central.

Para fins de comparação, ambas as técnicas serão abordadas no código a seguir:

### 1.1) Skip-Gram

In [31]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# Tokenizando as colunas
chats_clients['Pergunta_tokens'] = chats_clients['Pergunta'].apply(lambda x: word_tokenize(x.lower()))
chats_clients['Resposta_tokens'] = chats_clients['Resposta'].apply(lambda x: word_tokenize(x.lower()))

# Unindo as listas de tokens de Pergunta e Resposta para o modelo processar a lista
sentences_skip = chats_clients['Pergunta_tokens'].tolist() + chats_clients['Resposta_tokens'].tolist()

# Utilizando o modelo Word2Vec
model = Word2Vec(sentences_skip, vector_size=100, window=5, min_count=1, sg=1)  #sg=1 para Skip-Gram

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 1.2) Resultados Obtidos

In [32]:
# Acessando o vetor de uma palavra
vector_dia_skip = model.wv['dia']
print(vector_dia_skip)

[-0.23450153  0.5743515  -0.2622718  -0.17642635  0.00403905 -0.09535325
 -0.10779265  0.5065461  -0.01559812 -0.0432271  -0.3337096  -0.2850121
 -0.2152893   0.15000649 -0.07117829 -0.11381815  0.21159586 -0.08326062
  0.10552052 -0.18591234  0.27530658 -0.0064123   0.49890545  0.11552761
  0.4186123   0.35492542  0.00843004 -0.31771025 -0.09765682  0.28461465
  0.13181803  0.30639833  0.00255559  0.14311323  0.31003177  0.03480506
 -0.39424133 -0.27358612  0.02424719 -0.19503611  0.01369898 -0.31268996
 -0.00421823 -0.18180062  0.23319335 -0.05367816 -0.25725523 -0.03989065
  0.37093356 -0.20154352  0.07896322 -0.32546064 -0.21059947  0.1126088
 -0.27640614  0.31407648  0.24057718  0.0571409  -0.08398924 -0.07043582
  0.21160644  0.00286279  0.15212223  0.16359289  0.10691377 -0.1467504
  0.11067164  0.34942472 -0.2651137   0.21266246 -0.01751359 -0.15596253
 -0.01886441 -0.13220704  0.13362855  0.1744168   0.29857698 -0.02157015
 -0.3956314   0.07473525 -0.09149611 -0.2503021   0.16

In [33]:
# Encontrando palavras similares
similar_words_dia_skip = model.wv.most_similar('dia')
print(similar_words_dia_skip)

[('plantão', 0.7444322109222412), ('especial', 0.7364803552627563), ('defense', 0.696092426776886), ('bom', 0.6865017414093018), ('chamo', 0.6815490126609802), ('intelligence', 0.6792458891868591), ('processaremos', 0.6776232719421387), ('!', 0.6776218414306641), ('ontem', 0.6757254600524902), ('sim', 0.6731353998184204)]


In [34]:
# Encontrando palavras similares
similar_words_bom_skip = model.wv.most_similar('bom')
print(similar_words_bom_skip)

[('!', 0.7967286109924316), ('especial', 0.7068284153938293), ('processaremos', 0.7033593654632568), ('conforme', 0.6987793445587158), ('após', 0.6975905895233154), ('plantão', 0.6937530040740967), ('recuperar', 0.6906157732009888), ('chamo', 0.689026415348053), ('meus', 0.6877617239952087), ('dia', 0.6865018010139465)]


In [35]:
# Encontrando palavras similares
similar_words_dinheiro_skip = model.wv.most_similar('dinheiro')
print(similar_words_dinheiro_skip)

[('japão', 0.8701930642127991), ('enviam', 0.8534935712814331), ('belem', 0.8356502056121826), ('cair', 0.8277362585067749), ('poder', 0.8145753145217896), ('tempo', 0.8055461645126343), ('pro', 0.8054846525192261), ('pace', 0.8021837472915649), ('leva', 0.8018035292625427), ('pacing', 0.801596462726593)]


In [36]:
# Encontrando palavras similares
similar_words_pix_skip = model.wv.most_similar('pix')
print(similar_words_pix_skip)

[('sendo', 0.9820218086242676), ('via', 0.9648541808128357), ('feitos', 0.9589219689369202), ('realizadas', 0.9570752382278442), ('pagamentos', 0.9483535289764404), ('maioria', 0.9404789209365845), ('estão', 0.9317833185195923), ('pagas', 0.9111994504928589), ('apta', 0.9087498188018799), ('estiver', 0.9015684127807617)]


### 2.1) CBOW

In [37]:
# Unindo as listas de tokens de Pergunta e Resposta para o modelo processar a lista
sentences_CBOW = chats_clients['Pergunta_tokens'].tolist() + chats_clients['Resposta_tokens'].tolist()

# Utilizando o modelo Word2Vec
model_cbow = Word2Vec(sentences_CBOW, vector_size=100, window=5, min_count=1, sg=0)  #sg=0 para CBOW

### 2.2) Resultados Obtidos

In [38]:
# Acessando o vetor de uma palavra
vector_dia_cbow = model_cbow.wv['dia']
print(vector_dia_cbow)

[-0.13328609  0.40281826 -0.6347823   0.00686785  0.4329409  -0.45583832
 -0.39071977  1.6315885  -0.38337022 -0.3133451  -0.46551496 -0.25442097
 -0.5669697   0.47810513  0.08388798 -0.05075192  0.0380762   0.07873245
 -0.12069137 -0.44385445  0.685599   -0.12448116  0.62131894 -0.09491962
  0.22606473  0.15629455 -0.5643191  -0.6571369  -0.44250113  0.5861929
  0.39067557  0.1898819  -0.02813485 -0.39020848  0.0954349   0.3166498
 -0.45347992 -0.267827   -0.0818375   0.0618521   0.3110817  -0.35722402
 -0.22511959 -0.23361853  0.22947824  0.27233472 -0.48941165 -0.1386946
  0.2585106   0.08633156  0.66286725 -0.9218504  -0.53508544 -0.197806
  0.17991728 -0.00799364  0.34002402 -0.10845585 -0.0342777   0.03973758
 -0.16178936  0.0965642   0.1478521  -0.04529976 -0.14778751  0.02510194
  0.33844864  0.5878607  -0.98968357  0.5238548  -0.06792736 -0.02919173
  0.3945414  -0.37548795  0.43948713 -0.21689518  0.4384342   0.10400886
 -1.0314837  -0.09850072 -0.20840937 -0.19947672 -0.0178

In [39]:
# Encontrando palavras similares
similar_words_dia_cbow = model_cbow.wv.most_similar('dia')
print(similar_words_dia_cbow)

[('bom', 0.9799854159355164), ('olá', 0.9583070874214172), ('!', 0.9552152752876282), ('noite', 0.9512419104576111), ('informar', 0.9484986066818237), ('boa', 0.9479116201400757), ('tarde', 0.9377167224884033), ('feather', 0.9354929327964783), ('uma', 0.9345516562461853), ('aguardar', 0.9321815967559814)]


In [40]:
# Encontrando palavras similares
similar_words_bom_cbow = model_cbow.wv.most_similar('bom')
print(similar_words_bom_cbow)

[('dia', 0.9799852967262268), ('!', 0.9692397713661194), ('boa', 0.9667654037475586), ('noite', 0.9659730792045593), ('olá', 0.9623599648475647), ('feather', 0.9575132727622986), ('tarde', 0.9561945796012878), ('informar', 0.9405844807624817), ('processaremos', 0.9369285106658936), ('verificamos', 0.9365541934967041)]


In [41]:
# Encontrando palavras similares
similar_words_dinheiro_cbow = model_cbow.wv.most_similar('dinheiro')
print(similar_words_dinheiro_cbow)

[('japão', 0.9958521127700806), ('enviar', 0.9823628067970276), ('tempo', 0.9809801578521729), ('come', 0.9685292840003967), ('?', 0.9685205221176147), ('pra', 0.9679456353187561), ('meio', 0.9658198952674866), ('confirmar', 0.9653485417366028), ('levará', 0.9613340497016907), ('login', 0.9607809782028198)]


In [42]:
# Encontrando palavras similares
similar_words_pix_cbow = model_cbow.wv.most_similar('pix')
print(similar_words_pix_cbow)

[('via', 0.9910397529602051), ('feitos', 0.9785277247428894), ('sendo', 0.9651416540145874), ('pagamentos', 0.9597576856613159), ('estão', 0.9565461277961731), ('realizadas', 0.9519309997558594), ('pagas', 0.9085805416107178), ('chegam', 0.8789027333259583), ('dentro', 0.8596354126930237), ('apta', 0.8584651350975037)]


### 3.1) Word Embedding


Dessa forma, é fundamental gerar os embedding das colunas para que futuros modelos de aprendizado de máquina possam compreender e utilizar os dados de cada frase. Para tal, será utilizado uma média dos vetores de cada frase, desenvolvido a seguir, utilizando a abordagem CBOW (padrão):


In [43]:
import numpy as np

# Função para gerar o embedding médio de uma frase
def get_sentence_embedding(tokens, model):
    word_vectors = [model.wv[token] for token in tokens if token in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

### 3.2) Resultados Obtidos

In [44]:
# Aplicando a função para gerar os embeddings e salvando nas novas colunas
chats_clients['Pergunta_embedding'] = chats_clients['Pergunta_tokens'].apply(lambda tokens: get_sentence_embedding(tokens, model_cbow))
chats_clients['Resposta_embedding'] = chats_clients['Resposta_tokens'].apply(lambda tokens: get_sentence_embedding(tokens, model_cbow))

In [45]:
chats_clients['Pergunta_embedding'].head()

,Pergunta_embedding
0,"[-0.0845738, 0.4517601, -0.2336668, -0.0883541..."
1,"[-0.25928864, 0.63610363, -0.40346727, -0.2502..."
2,"[-0.23365983, 0.67679834, -0.3792954, -0.14951..."
3,"[0.01179275, 0.421502, -0.24553484, -0.0747979..."
4,"[-0.06969267, 0.5215444, -0.21788402, -0.07733..."


In [46]:
chats_clients['Resposta_embedding'].head()

,Resposta_embedding
0,"[-0.07603086, 0.38741657, -0.248062, -0.051126..."
1,"[-0.10161663, 1.1046908, -0.32487667, -0.25247..."
2,"[-0.14437574, 0.52948046, -0.3372583, -0.11305..."
3,"[-0.0010399645, 0.46549717, -0.16220675, 0.059..."
4,"[-0.0010399645, 0.46549717, -0.16220675, 0.059..."


## 2) Base de Dados Final

In [47]:
chats_clients

,No,Intencao,Pergunta,Resposta,Pergunta_tokens,Resposta_tokens,Pergunta_embedding,Resposta_embedding
0,1,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aq...,"Bom dia! Sim, o sr pode utilizar o cartão de d...","[boa, dia.tudo, bem, ?, eu, gostaria, de, sabe...","[bom, dia, !, sim, ,, o, sr, pode, utilizar, o...","[-0.0845738, 0.4517601, -0.2336668, -0.0883541...","[-0.07603086, 0.38741657, -0.248062, -0.051126..."
1,2,Como fazer remessa,Como enviar dinheiro do Japão?,"Para se inscrever no serviço de remessa, por f...","[como, enviar, dinheiro, do, japão, ?]","[para, se, inscrever, no, serviço, de, remessa...","[-0.25928864, 0.63610363, -0.40346727, -0.2502...","[-0.10161663, 1.1046908, -0.32487667, -0.25247..."
2,3,Tempo de remessa,Quanto tempo levará para o beneficiário recebe...,"Via de regra, as remessas serão pagas via PIX ...","[quanto, tempo, levará, para, o, beneficiário,...","[via, de, regra, ,, as, remessas, serão, pagas...","[-0.23365983, 0.67679834, -0.3792954, -0.14951...","[-0.14437574, 0.52948046, -0.3372583, -0.11305..."
3,4,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer a transferência de ...,iremos processar a sua solicitacao.\nMuito obr...,"[boa, tarde, acabei, de, fazer, a, transferênc...","[iremos, processar, a, sua, solicitacao, ., mu...","[0.01179275, 0.421502, -0.24553484, -0.0747979...","[-0.0010399645, 0.46549717, -0.16220675, 0.059..."
4,5,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB ...,iremos processar a sua solicitacao.\nMuito obr...,"[poderia, fazer, a, remessa, de, 22yenes, para...","[iremos, processar, a, sua, solicitacao, ., mu...","[-0.06969267, 0.5215444, -0.21788402, -0.07733...","[-0.0010399645, 0.46549717, -0.16220675, 0.059..."
...,...,...,...,...,...,...,...,...
1510,501,Atualizacao de dados cadastrais,boa tarde como faço paratrooper fazer a mudanç...,vitamin A senhora pode trocar pelo próprio app...,"[boa, tarde, como, faço, paratrooper, fazer, a...","[vitamin, a, senhora, pode, trocar, pelo, próp...","[-0.045387324, 0.4871446, -0.25954545, -0.1027...","[-0.08179023, 0.5393536, 0.06090493, -0.130234..."
1511,502,Tempo de entrega do cartao,Bom dia Consegui fazer depósito sem group O ca...,feather boa noite O sr solicitou dia 28. Será ...,"[bom, dia, consegui, fazer, depósito, sem, gro...","[feather, boa, noite, o, sr, solicitou, dia, 2...","[-0.09042335, 0.4562718, -0.2477673, -0.065512...","[-0.07728403, 0.36206418, -0.19521685, -0.0154..."
1512,503,Cancelamento,"feather boa noite,a beneficiária me avisou que...","Sim, podemos cancelar O valiancy voltará no se...","[feather, boa, noite, ,, a, beneficiária, me, ...","[sim, ,, podemos, cancelar, o, valiancy, volta...","[-0.09668481, 0.5270748, -0.25122532, -0.10119...","[-0.12857951, 0.5221245, -0.28093454, -0.12145..."
1513,504,Reembolso,"Boa noite Europe fiz um depósito errado, era p...",É possível solicitar o reembolso diretamente d...,"[boa, noite, europe, fiz, um, depósito, errado...","[é, possível, solicitar, o, reembolso, diretam...","[-0.08569564, 0.53608567, -0.27029905, -0.1037...","[-0.1064237, 0.4818608, -0.13950056, -0.028333..."


In [83]:
from sklearn.metrics import classification_report

class IntentClassifier:
    def __init__(self, epochs=180, batch_size=16, test_size=0.4, validation_split=0.4, learning_rate=0.001):
        self.epochs = epochs
        self.batch_size = batch_size
        self.test_size = test_size
        self.validation_split = validation_split
        self.learning_rate = learning_rate
        self.model = None
        self.label_encoder = None

    def prepare_data(self, chats_clients):
        answer_embeddings = np.array(chats_clients['Pergunta_embedding'].tolist())
        response_embeddings = np.array(chats_clients['Resposta_embedding'].tolist())

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(chats_clients['Intencao'])

        combined_embeddings = np.stack((answer_embeddings, response_embeddings), axis=1)
        X_train, X_test, y_train, y_test = train_test_split(combined_embeddings, self.labels, test_size=self.test_size, random_state=42)

        return X_train, X_test, y_train, y_test

    def build_model(self, input_shape):
        self.model = Sequential()
        self.model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
        self.model.add(Dropout(0.5))
        self.model.add(LSTM(64))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(32, activation='relu'))
        self.model.add(Dense(len(np.unique(self.labels)), activation='softmax'))
        optimizer = Adam(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.model.summary()

    def train(self, X_train, X_test, y_train, y_test):
        early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
        model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')

        self.model.fit(X_train, y_train, epochs=self.epochs,
                       batch_size=self.batch_size,
                       validation_split=self.validation_split,
                       callbacks=[
                           early_stopping,
                           model_checkpoint
                           ])

        loss, accuracy = self.model.evaluate(X_test, y_test)
        print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)

        return y_pred_classes

    def generate_report(self, y_test, y_pred_classes):

      print(len(y_test))
      print(classification_report(y_test, y_pred_classes))

      ### Entender o problema abaixo

        # precision, recall, f1, support = precision_recall_fscore_support(y_test, y_pred_classes)

        # report_df = pd.DataFrame({
        #     'Class': self.label_encoder.classes_,
        #     'Precision': precision,
        #     'Recall': recall,
        #     'F1-Score': f1,
        #     'Support': support
        # })

        # report_df.loc['Average'] = report_df[['Precision', 'Recall', 'F1-Score', 'Support']].mean()
        # report_df.loc['Average', 'Class'] = 'Average'

        # print(report_df)


In [84]:
classifier = IntentClassifier()
X_train, X_test, y_train, y_test = classifier.prepare_data(chats_clients)
classifier.build_model(input_shape=(X_train.shape[1], X_train.shape[2]))
classifier.train(X_train, X_test, y_train, y_test)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_9 (Bidirectional)      │ (None, 2, 256)              │         234,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 2, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ (None, 64)                  │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 18)                  │             594 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 323,506 (1.23 MB)

 Trainable params: 323,506 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/180
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.1239 - loss: 2.7712 - val_accuracy: 0.2143 - val_loss: 2.4585
Epoch 2/180
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2936 - loss: 2.4684 - val_accuracy: 0.4505 - val_loss: 1.9852
Epoch 3/180
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4501 - loss: 2.0368 - val_accuracy: 0.4478 - val_loss: 1.7970
Epoch 4/180
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4713 - loss: 1.7662 - val_accuracy: 0.4835 - val_loss: 1.7475
Epoch 5/180
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4844 - loss: 1.6980 - val_accuracy: 0.5110 - val_loss: 1.6061
Epoch 6/180
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4991 - loss: 1.6481 - val_accuracy: 0.5302 - val_loss: 1.6237
Epoch 7/180
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5475 - loss: 1.5358 - val_accuracy: 0.5027 - val_loss: 1.6049
Epoch 8/180
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5095 - loss: 1.6255 - val_accuracy: 0.5

In [82]:
y_pred_classes = classifier.predict(X_test)
classifier.generate_report(y_test, y_pred_classes)

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
              precision    recall  f1-score   support

           0       0.35      0.32      0.33        19
           1       0.30      0.43      0.35         7
           2       0.44      0.67      0.53        27
           3       0.33      0.50      0.40        10
           4       0.55      0.57      0.56        21
           5       0.71      0.31      0.43        16
           6       0.84      0.94      0.89        83
           7       0.89      0.92      0.90       154
           8       0.00      0.00      0.00         2
           9       0.84      0.97      0.90        70
          10       0.69      0.75      0.72        24
          11       0.33      0.27      0.30        11
          12       0.75      0.28      0.41        43
          13       0.00      0.00      0.00         0
          14       0.21      0.24      0.22        25
          15       0.50      0.53      0.51        19
          16       0.79      0.80      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
